<a href="https://colab.research.google.com/github/booluckgmie/podac/blob/main/Dashboard_Boundary_demo_by_DUNv2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas_geojson geopandas folium panel

All in features with selection

In [3]:
import folium as fm
import panel as pn
from pandas_geojson import filter_geojson

def create_app2(sociodemo, geo_json, select):
    selectv = select.value
    filter = [selectv]
    selectgeo = filter_geojson(geo_json=geo_json, filter_list=filter, property_key='dun')

    # create map object
    map = fm.Map(location=[5.016806622688496, 107.9456777002687], tiles='cartodbpositron', zoom_start=6)
    fm.GeoJson(selectgeo, name='DUN boundary', tooltip=fm.GeoJsonTooltip(fields=['dun', 'parlimen'])).add_to(map)
    map.fit_bounds(map.get_bounds(), padding=(30, 30))
    folium_map = pn.panel(map, height=400)

    # select data based on selected value
    select_sociodemo = sociodemo[sociodemo['dun'] == selectv].copy()
    select_sociodemo.loc[:, 'watercover'] = select_sociodemo['watercover'].round(decimals=3)
    max_elev = select_sociodemo['max_elevation'].iloc[0]
    water = select_sociodemo['watercover'].iloc[0]
    pop = select_sociodemo['population_total'].iloc[0]

    # create indicators
    number = pn.indicators.Number(format='{value}')
    indicator = pn.Row(
        number.clone(name='Population', value=pop, colors=[(33, 'black')]),
        number.clone(name='Maximum Elevation', value=max_elev, colors=[(66, 'blue')]),
        number.clone(name='Surface covered in water', value=water, colors=[(100, 'red')]),
        sizing_mode='stretch_width'
    )

    # create description
    description = select_sociodemo['dun'].iloc[0]
    str_pane = pn.indicators.Number(name=str(description))

    # create panel object
    app2 = pn.Column(
        pn.Column(
            str_pane,
            indicator,
            folium_map,
            sizing_mode='scale_both'
        ),
        sizing_mode='scale_both'
    )

    return app2

import json
import urllib.request
import geopandas as gpd
import pandas as pd

url = 'https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/electoral_1_dun.geojson'
geo_json = json.load(urllib.request.urlopen(url))

duns = gpd.read_file("https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/electoral_1_dun.geojson")

sociodemo = pd.read_csv('https://github.com/booluckgmie/podac/raw/main/sociodemo.csv')

# READ DATAFRAME
filter = ['Pulau Pinang']
select_options = duns[duns['state'].isin(filter)]['dun'].values.tolist()

# CREATE SELECTION
select = pn.widgets.Select(name='Select DUN', options=select_options)

def on_select_change(event):
    layout[1] = create_app2(sociodemo, geo_json, select)

select.param.watch(on_select_change, "value")

app2 = create_app2(sociodemo, geo_json, select)

# Arrange the widgets and apps
layout = pn.Column(select, app2)
layout.save('app.html', embed=True)

layout.servable()



Column
    [0] Select(options=['N.01 Penaga', ...], value='N.01 Penaga')
    [1] Column(sizing_mode='scale_both')
        [0] Column(sizing_mode='scale_both')
            [0] Number(name='N.01 Penaga')
            [1] Row(sizing_mode='stretch_width')
                [0] Number(colors=[(33, 'black')], name='Population', value=28207)
                [1] Number(colors=[(66, 'blue')], name='Maximum Elevation', value=31.0)
                [2] Number(colors=[(100, 'red')], name='Surface covered i..., value=0.633)
            [2] Folium(Map, height=400)